In [1]:
import torch
import torchvision
from chatglm1 import *
from chatglm2 import *
import pandas as pd
import json
import time
import faiss
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from Dataset_improved import *
# from chains.local_doc_qa import LocalDocQA
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.docstore.base import AddableMixin, Docstore
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score,precision_recall_curve
# 加载原始数据
raw_data = read_json_file("/mnt/workspace/pangtianqi/medical_kb_chatbot/data/test_demo.json")
testing_dataset = MyDataset(raw_data)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=1, shuffle=False)

/home/pai/envs/kb_chat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-19 06:38:55,758] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
#labels_recall ->所有label存在一个列表
#labels ->按照问题划分的label列表
testing_text = []
labels_list = []
labels_list_recall = []
for batch_data in testing_dataloader:
    data, label = batch_data[2:]
    for idx in data:
        for content in idx:
            testing_text.append(content)
    labels_list_recall.append(label)
    for i in label:
        labels_list.append(i)
    # print(labels_list)
labels_recall = [torch.cat(tensor).tolist() for tensor in labels_list_recall]
labels = [tensor.item() for tensor in labels_list]
#print("testing_text",testing_text)
# print("label_recall",labels_recall)

In [3]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('/mnt/workspace/pangtianqi/medical_kb_chatbot/moka-ai/m3e-base', device='cuda:0')
content_list = []
for batch_data in testing_text:
    # data, label = batch_data[2:]
    content_vectors = encoder.encode(batch_data)
    content_list.append(content_vectors)

vectors = np.array(content_list,dtype=np.float32)

In [7]:
import torch
import torchvision.transforms as transforms

# 创建数据增强的变换操作
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),  # 随机水平翻转
#     transforms.RandomVerticalFlip(),    # 随机垂直翻转
#     transforms.RandomRotation(10),      # 随机旋转（角度范围为-10到10）
#     transforms.RandomResizedCrop(224),  # 随机裁剪和缩放到指定大小
#     transforms.ToTensor(),              # 转换为Tensor对象
# ])


augmented_data = np.flip(vectors, axis=1)
# data = torch.randn(3, 256, 256)  # 假设原始数据是一个3通道的256x256图像

# 进行数据增强
# augmented_data = transform(vectors)

# 显示原始数据和增强后的数据
print("原始数据形状:", vectors)
print("增强后的数据形状:", augmented_data)

原始数据形状: [[ 0.40726885  0.25266618  0.5639562  ... -0.82608134 -0.44088337
  -0.2532817 ]
 [ 0.4660251   0.24446245  0.30897647 ... -1.002208   -0.33304214
  -0.03854139]
 [ 0.54311955  0.5339352   0.5594772  ... -0.9275461  -0.37917095
  -0.17953338]
 ...
 [ 0.25744325  0.23370641 -0.016852   ... -0.32391912 -0.47256005
  -0.7664421 ]
 [-0.13081086  0.4449784   0.4029541  ... -0.7584297  -0.72457683
  -0.79325604]
 [ 0.01894369  0.626241    0.5815307  ... -0.19894889 -1.1303027
  -1.1365415 ]]
增强后的数据形状: [[-0.2532817  -0.44088337 -0.82608134 ...  0.5639562   0.25266618
   0.40726885]
 [-0.03854139 -0.33304214 -1.002208   ...  0.30897647  0.24446245
   0.4660251 ]
 [-0.17953338 -0.37917095 -0.9275461  ...  0.5594772   0.5339352
   0.54311955]
 ...
 [-0.7664421  -0.47256005 -0.32391912 ... -0.016852    0.23370641
   0.25744325]
 [-0.79325604 -0.72457683 -0.7584297  ...  0.4029541   0.4449784
  -0.13081086]
 [-1.1365415  -1.1303027  -0.19894889 ...  0.5815307   0.626241
   0.01894369]]


In [2]:
import numpy as np

def compute_kl_divergence(p, q):
    kl_divergence = np.sum(p * np.log(p / q))
    return kl_divergence

# 示例：计算两个离散概率分布的KL散度
p = np.array([0.2, 0.3, 0.5])
q = np.array([0.1, 0.4, 0.5])

kl_divergence = compute_kl_divergence(p, q)
print("KL散度:", kl_divergence)

KL散度: 0.05232481437645474


In [ ]:
#给q再加上关键词 